In [1]:
import torch
import MobileNetV2Pretrained
import sys
import os
ROOT_DIR = os.path.abspath(os.path.join(os.getcwd(), ".."))
sys.path.append(ROOT_DIR)
import utils.helper as helper
import utils.qai_hub_jobs as qai_hub_jobs
import utils.tfhelper as tfhelper
import qai_hub as hub
import aimet_torch

2025-02-18 12:46:03,916 - root - INFO - AIMET


c:\Users\Pro\Documents\___FAKULTET___\LPCV\LPCV_2025_T1\LPCV_2025_T1\venv\Lib\site-packages\onnxscript\converter.py:823: FutureWarning: 'onnxscript.values.Op.param_schemas' is deprecated in version 0.1 and will be removed in the future. Please use '.op_signature' instead.
  param_schemas = callee.param_schemas()
<frozen abc>:106: FutureWarning: `NLLLoss2d` has been deprecated. Please use `NLLLoss` instead as a drop-in replacement and see https://pytorch.org/docs/main/nn.html#torch.nn.NLLLoss for more details.


In [2]:
pretrained_model = MobileNetV2Pretrained.PreprocessedMobileNetV2(64, "model/mobilenet_v2_coco.pth")

In [30]:
pretrained_model.eval()
input_shape: tuple[int, ...] = (1, 3, 224, 224)
example_input = torch.rand(input_shape)
pt_model = torch.jit.trace(pretrained_model, example_input)

# # Compile model on a specific device
# compile_job = hub.submit_compile_job(
#     pt_model,
#     name="reference_model", # Replace with your model name
#     device=hub.Device("Snapdragon 8 Elite QRD"),
#     input_specs=dict(image=input_shape),
# )

# compiled_model = compile_job.get_target_model()


In [12]:
profile_job = qai_hub_jobs.profile_job(compiled_model)

Scheduled profile job (jp4nv88vg) successfully. To see the status and results:
    https://app.aihub.qualcomm.com/jobs/jp4nv88vg/



In [ ]:
profile_job.download_results("profile_results")

In [31]:
from aimet_common.defs import QuantScheme
from aimet_common.quantsim_config.utils import get_path_for_per_channel_config
from aimet_torch.quantsim import QuantizationSimModel

sim = QuantizationSimModel(pretrained_model, 
                           example_input,
                           quant_scheme=QuantScheme.post_training_tf,
                           config_file=get_path_for_per_channel_config(),
                           default_param_bw=8,
                           default_output_bw=8)
print(sim)

2025-02-18 13:35:25,747 - Quant - INFO - Unsupported op type Squeeze
2025-02-18 13:35:25,748 - Quant - INFO - Unsupported op type Mean
2025-02-18 13:35:25,754 - Quant - INFO - Selecting DefaultOpInstanceConfigGenerator to compute the specialized config. hw_version:default
-------------------------
Quantized Model Report
-------------------------
PreprocessedMobileNetV2(
  (mobilenet_v2): MobileNetV2(
    (features): Sequential(
      (0): Conv2dNormActivation(
        (0): QuantizedConv2d(
          3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False
          (param_quantizers): ModuleDict(
            (weight): QuantizeDequantize(shape=(32, 1, 1, 1), qmin=-128, qmax=127, symmetric=True)
          )
          (input_quantizers): ModuleList(
            (0): QuantizeDequantize(shape=(), qmin=0, qmax=255, symmetric=False)
          )
          (output_quantizers): ModuleList(
            (0): None
          )
        )
        (1): QuantizedBatchNorm2d(
          32, ep

In [32]:

sim = QuantizationSimModel(pretrained_model, 
                           example_input,
                           quant_scheme=QuantScheme.training_range_learning_with_tf_init,
                           config_file=get_path_for_per_channel_config(),
                           default_param_bw=8,
                           default_output_bw=8)
print(sim)

2025-02-18 13:35:35,519 - Quant - INFO - Unsupported op type Squeeze
2025-02-18 13:35:35,520 - Quant - INFO - Unsupported op type Mean
2025-02-18 13:35:35,526 - Quant - INFO - Selecting DefaultOpInstanceConfigGenerator to compute the specialized config. hw_version:default
-------------------------
Quantized Model Report
-------------------------
PreprocessedMobileNetV2(
  (mobilenet_v2): MobileNetV2(
    (features): Sequential(
      (0): Conv2dNormActivation(
        (0): QuantizedConv2d(
          3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False
          (param_quantizers): ModuleDict(
            (weight): QuantizeDequantize(shape=(32, 1, 1, 1), qmin=-128, qmax=127, symmetric=True)
          )
          (input_quantizers): ModuleList(
            (0): QuantizeDequantize(shape=(), qmin=0, qmax=255, symmetric=False)
          )
          (output_quantizers): ModuleList(
            (0): None
          )
        )
        (1): QuantizedBatchNorm2d(
          32, ep

In [ ]:
# sim.model(torch.rand(input_shape))

RuntimeError: Failed to run QuantizeDequantize since quantization parameters are not initialized. Please initialize the quantization parameters using `compute_encodings()`.

In [33]:
def forward_pass(model):
    with torch.no_grad():
        model(torch.randn((1, 3, 224, 224)))

sim.compute_encodings(forward_pass)

# output = sim.model()


In [34]:
sim.model(torch.rand(input_shape))

DequantizedTensor([[ 0.1940, -0.4850, -0.6628, -0.3071, -1.0184, -0.7759,
                    -0.0808,  0.7274,  1.2286,  0.5011, -0.5011, -0.4365,
                     2.0530, -0.3395, -0.1778, -0.4203,  0.6951, -0.5011,
                    -0.2263, -0.2586,  0.3718,  0.0162,  0.0485,  0.0970,
                     0.1455,  0.7759,  0.1940,  3.0714, -0.0162,  0.4526,
                     0.7436, -0.2425,  0.0485,  0.3556, -0.5011,  0.2748,
                     0.0970,  0.4041, -0.9538, -0.5335, -0.8406, -0.7274,
                     0.1455, -0.2263, -0.2910, -0.9861, -0.4203,  0.0323,
                     0.0647,  0.2101,  0.3071,  0.5496,  0.2586,  0.3556,
                    -0.3718, -0.3718,  0.0485, -0.5011,  0.4041, -0.6304,
                     1.7620,  0.3071,  0.0323,  0.0808]],
                  grad_fn=<AliasBackward0>)

In [ ]:
# for param in sim.model.parameters():
#     print(type(param))

In [35]:
os.makedirs("model/quantized", exist_ok=True)
sim.export(path = "model/quantized", filename_prefix = "mobilenet_v2_coco_quantized", dummy_input=example_input)

2025-02-18 13:37:47,655 - Utils - INFO - successfully created onnx model with 140/222 node names updated
2025-02-18 13:37:47,752 - Quant - WARNING - The following layers were not found in the exported onnx model. Encodings for these layers will not appear in the exported encodings file, however it will continue to exist in torch encoding file:
['mobilenet_v2.classifier.0']
This can be due to several reasons:
	- The layer is set to quantize with float datatype, but was not exercised in compute encodings. Not an issue if the layer is not meant to be run.
	- The layer has valid encodings but was not seen while exporting to onnx using the dummy input provided in sim.export(). Ensure that the dummy input covers all layers.
2025-02-18 13:37:47,752 - Quant - INFO - Layers excluded from quantization: []
2025-02-18 13:37:47,754 - Quant - WARNING - Quantsim export will stop exporting encodings for saving and loading in a future AIMET release.
To export encodings for saving and loading, use Quant

In [11]:
cj = hub.submit_compile_job(
    model = "model/quantized/quantized.aimet",
    name="quantized_model", # Replace with your model name
    device=hub.Device("Snapdragon 8 Elite QRD"),
    options="--target_runtime qnn_lib_aarch64_android --quantize_full_type int8",
)

Uploading quantized.aimet.zip
2025-02-18 12:58:33,859 - root - INFO - Uploading asset to https://tetrahub-qprod-userdata.s3-accelerate.amazonaws.com/models/mq9wzp70m_Y2H3Zu7belpwK0Vu.aimet.zip?uploadId=U4fgA.2_5fOWMIyzvqPpOiuuKYP0g.lF0oTZwwNRhu5CVlcIgGqtDIK0uyKuWam11c2v9IDmeIZXLVNkyidolcwNEJxwZvPmSwPi37gxsDTfGhYSK0H0oASdcd8bZKtO6m5aYno6l1yTVbP27nUbIrzyn2SSxAHk4HFwKIy2I3I-&partNumber=1&A


100%|██████████| 5.08M/5.08M [00:01<00:00, 3.62MB/s]

2025-02-18 12:58:35,335 - root - INFO - Successfully uploaded asset with response status: 200


Scheduled compile job (jp12e332g) successfully. To see the status and results:
    https://app.aihub.qualcomm.com/jobs/jp12e332g/



In [13]:
pf = qai_hub_jobs.profile_job(cj.get_target_model())

Scheduled profile job (jgn03xxr5) successfully. To see the status and results:
    https://app.aihub.qualcomm.com/jobs/jgn03xxr5/



In [ ]:
import dataset.utils as dsutils
import utils.input_getter as ig

download_path = tfhelper.download_model_from_compile_job(compile, 'ref_model')
TFModel = tfhelper.TFHelper(download_path)

input = ig.mug_image_getter()

helper.print_probablities_from_output(TFModel.run_inference(input.get_input_numpy()), categories=dsutils.GLOBAL_CLASSES, modelname = 'TensorFlow')